In [8]:
import pandas as pd
import ast  

AttributeError: partially initialized module 'pandas' has no attribute 'core' (most likely due to a circular import)

In [ ]:
df_movie_meta= pd.read_csv('Data/IMDB/movies_metadata.csv')
df_movie_rating= pd.read_csv('Data/IMDB/ratings.csv')
df_movie_action= pd.read_csv('Data/TMDB/action.csv')
df_movie_ad= pd.read_csv('Data/TMDB/adventure.csv')
df_movie_animation= pd.read_csv('Data/TMDB/animation.csv')
df_movie_biography= pd.read_csv('Data/TMDB/biography.csv')
df_movie_crime= pd.read_csv('Data/TMDB/crime.csv')
df_movie_family= pd.read_csv('Data/TMDB/family.csv')
df_movie_fantasy= pd.read_csv('Data/TMDB/fantasy.csv')
df_movie_film_noir= pd.read_csv('Data/TMDB/film-noir.csv')
df_movie_history= pd.read_csv('Data/TMDB/history.csv')
df_movie_horror= pd.read_csv('Data/TMDB/horror.csv')
df_movie_mystery= pd.read_csv('Data/TMDB/mystery.csv')
df_movie_romance= pd.read_csv('Data/TMDB/romance.csv')
df_movie_scifi= pd.read_csv('Data/TMDB/scifi.csv')
df_movie_sports= pd.read_csv('Data/TMDB/sports.csv')
df_movie_thriller= pd.read_csv('Data/TMDB/thriller.csv')
df_movie_war= pd.read_csv('Data/TMDB/war.csv')
df_tmdb_links = pd.read_csv('Data/IMDB/links.csv')

In [ ]:
# Set the option to display all columns
pd.set_option('display.max_columns', None)

# Display the first few rows of the DataFrame
df_movie_meta.head()

In [ ]:

# List of DataFrames
dfs = [
    df_movie_action, df_movie_ad, df_movie_animation, df_movie_biography,
    df_movie_crime, df_movie_family, df_movie_fantasy, df_movie_film_noir,
    df_movie_history, df_movie_horror, df_movie_mystery, df_movie_romance,
    df_movie_scifi, df_movie_sports, df_movie_thriller, df_movie_war
]

# Merge DataFrames with full outer join on 'id' and 'movie_id'
df_merged_movies = pd.concat(dfs, axis=0, join='outer', ignore_index=True, sort=False, keys=['id', 'movie_id'])




In [ ]:
df_merged_movies.isna().sum()

In [ ]:
df_merged_movies.head()

In [ ]:
df_merged_movies['gross(in $)'].isna().sum()

In [ ]:
# Assuming you have a DataFrame 'df'
df_merged_movies.rename(columns={'gross(in $)': 'revenue'}, inplace=True)


In [ ]:
df_merged_movies.shape

In [ ]:
df_merged_movies['movie_id'] = pd.to_numeric(df_merged_movies['movie_id'].str.replace('tt', ''), errors='coerce')

In [ ]:
df_merged_movies.info()

In [ ]:
df_merged_movies.head()

In [ ]:
df_merged_movies['revenue'].count()

In [ ]:
df_tmdb_links.head()

In [ ]:
df_tmdb_links.isna().sum()

In [ ]:
tmdb_df = pd.merge(df_tmdb_links, df_merged_movies, left_on='tmdbId', right_on='movie_id', how='right')

In [ ]:
tmdb_df.head()

In [ ]:
tmdb_df.isna().sum()

In [ ]:
df_movie_meta.head()


In [ ]:
df_movie_meta['imdbId'] = pd.to_numeric(df_movie_meta['imdbId'].str.replace('tt', ''), errors='coerce')


In [ ]:
df_movie_meta.rename(columns={'genres': 'genre', 'imdb_id':'imdbId'}, inplace=True)

In [ ]:
df_movie_meta.head()

In [ ]:
movie_df = pd.merge(df_movie_meta, tmdb_df, left_on='imdb_id', right_on='imdbId', how='outer')

In [ ]:
# movie_df = df_movie_meta.combine_first(tmdb_df)

In [ ]:
movie_df.tail()

In [ ]:
movie_df.shape

In [ ]:
df_movie_meta.isna().sum()

In [ ]:
tmdb_df.isna().sum()

In [ ]:
tmdb_df.shape

In [ ]:
movie_df.isna().sum()

In [ ]:
movie_df.head()

In [ ]:
movie_df.video.value_counts()

In [ ]:
movie_df.status.value_counts()

In [ ]:
movie_df.tagline[1]

In [ ]:
movie_df.homepage.value_counts()

In [ ]:
movie_df.drop(columns=["video","poster_path","homepage","overview","tagline"], inplace=True)

In [ ]:
movie_df.head(5)

In [ ]:
movie_df["belongs_to_collection"][0]

In [ ]:
# Assuming the 'belongs_to_collection' column contains strings that look like dictionaries
movie_df['belongs_to_collection'] = movie_df['belongs_to_collection'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else None)

# Create new columns for 'collection_id' and 'collection_name'
movie_df['collection_id'] = movie_df['belongs_to_collection'].apply(lambda x: x['id'] if isinstance(x, dict) else None)
movie_df['collection_name'] = movie_df['belongs_to_collection'].apply(lambda x: x['name'] if isinstance(x, dict) else None)
movie_df.drop(columns=['belongs_to_collection'], inplace=True)



In [ ]:
movie_df.head()

In [ ]:
movie_df.drop(columns=['belongs_to_collection'], inplace=True)

In [ ]:
movie_df["spoken_languages"][3]

In [ ]:
def extract_and_create_language_columns(df):
    # Define a custom function to extract language names
    def extract_language_name(lang_data):
        try:
            lang_list = ast.literal_eval(lang_data)  # Convert the string to a list of dictionaries
            return [lang['name'] for lang in lang_list]  # Extract the 'name' values
        except (ValueError, SyntaxError):
            return None

    # Apply the custom function to create a new column 'spoken_languages_names'
    df['spoken_languages_names'] = df['spoken_languages'].apply(extract_language_name)

    # Create 'spoken_language_1' and 'spoken_language_2' columns
    df['spoken_language_1'] = df['spoken_languages_names'].apply(lambda x: x[0] if isinstance(x, list) and len(x) > 0 else None)
    df['spoken_language_2'] = df['spoken_languages_names'].apply(lambda x: x[1] if isinstance(x, list) and len(x) > 1 else None)

    # Drop the 'spoken_languages_names' column if desired
    df.drop(columns=['spoken_languages_names'], inplace=True)

    return df

# Example usage:
movie_df = extract_and_create_language_columns(movie_df)


In [ ]:
movie_df.drop(columns=['spoken_languages'], inplace=True)


In [ ]:
movie_df.head()

In [ ]:
movie_df['production_companies'][1]

In [ ]:
def extract_company_data(company_data):
    try:
        company_list = ast.literal_eval(company_data)  # Convert the string to a list of dictionaries
        return company_list
    except (ValueError, SyntaxError):
        return None

# Apply the custom function to create a new column 'production_companies_data'
movie_df['production_companies_data'] = movie_df['production_companies'].apply(extract_company_data)

# Determine the maximum number of production companies in a row
max_companies = movie_df['production_companies_data'].apply(lambda x: len(x) if isinstance(x, list) else 0).max()

# Create separate columns for production company names and IDs
for i in range(max_companies):
    movie_df[f'production_company_{i+1}_name'] = movie_df['production_companies_data'].apply(
        lambda x: x[i]['name'] if isinstance(x, list) and len(x) > i else None)
    movie_df[f'production_company_{i+1}_id'] = movie_df['production_companies_data'].apply(
        lambda x: x[i]['id'] if isinstance(x, list) and len(x) > i else None)

# Drop the 'production_companies_data' column if desired
movie_df.drop(columns=['production_companies_data'], inplace=True)
movie_df.drop(columns=["production_companies"], inplace=True)


In [ ]:
# Define a custom function to extract country data
def extract_country_data(country_data):
    try:
        country_list = ast.literal_eval(country_data)  # Convert the string to a list of dictionaries
        return country_list
    except (ValueError, SyntaxError):
        return None

# Apply the custom function to create a new column 'production_countries_data'
movie_df['production_countries_data'] = movie_df['production_countries'].apply(extract_country_data)

# Determine the maximum number of production countries in a row
max_countries = movie_df['production_countries_data'].apply(lambda x: len(x) if isinstance(x, list) else 0).max()

# Create separate columns for production country names
for i in range(max_countries):
    movie_df[f'production_country_{i+1}_name'] = movie_df['production_countries_data'].apply(
        lambda x: x[i]['name'] if isinstance(x, list) and len(x) > i else None)

# Drop the 'production_countries_data' column if desired
movie_df.drop(columns=['production_countries_data'], inplace=True)
movie_df.drop(columns=["production_countries"], inplace=True)

In [ ]:
movie_df.genres[0]

In [ ]:
def extract_genre_data(genre_data):
    try:
        genre_list = ast.literal_eval(genre_data)  # Convert the string to a list of dictionaries
        return [genre['name'] for genre in genre_list]  # Extract the 'name' values
    except (ValueError, SyntaxError):
        return None

# Apply the custom function to create a new column 'genre_data'
movie_df['genre_data'] = movie_df['genres'].apply(extract_genre_data)

# Get a list of all unique genres
unique_genres = set(genre for genres in movie_df['genre_data'] if isinstance(genres, list) for genre in genres)

# Create a new column for each genre and set values as 1 or 0
for genre in unique_genres:
    movie_df[genre] = movie_df['genre_data'].apply(lambda x: 1 if isinstance(x, list) and genre in x else 0)

# Drop the 'genre_data' column if desired
movie_df.drop(columns=['genre_data'], axis=1, inplace=True)
movie_df.drop(columns=["genres"], inplace=True)

In [ ]:
movie_df.head()